### Prerequisite

In [2]:
# !pip install langchain
# !pip install chromadb
# !pip install sentence-transformers

In [1]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser

from langchain.storage._lc_store import create_kv_docstore

In [2]:
import chromadb
from chromadb.config import Settings

指定Chromadb client

In [13]:
#連線設定
httpClient = chromadb.HttpClient(
  host='64.176.47.89', port=8000,
  settings=Settings(chroma_client_auth_provider="basic",chroma_client_auth_credentials="admin:admin")
  )

collections = httpClient.list_collections() #資料庫列表
collections

[Collection(name=xt131028_v),
 Collection(name=xt131028),
 Collection(name=simple_html_Stella_Base_zh_v3_1792d_t13_aia_with_qa),
 Collection(name=simple_html_nomic_embed_text_v1_f16_t5),
 Collection(name=X131010),
 Collection(name=my_collection),
 Collection(name=simple_html_nomic_embed_text_v1_f16_t6_with_qa),
 Collection(name=xt131028_v1),
 Collection(name=xt131028_v1.1),
 Collection(name=simple_html_nomic_embed_text_v1_f16_t8_aia_with_qa)]

In [14]:
import json
import sqlite3
from typing import Tuple, Iterator
from typing import Sequence, List, Optional
from langchain.schema import BaseStore
from pypika import Query, Table, Field, Column

指定做為docstore的sqlite3資料庫檔案的路徑

GoogleDrive下載:

https://drive.google.com/drive/folders/1h5y150ObqGHXhxAkVWOqvgt9U3MUXhE9?usp=sharing


．chroma_t5.sqlite3 : 抓取AIA官網下的所有簡章html  
．chroma_t6_with_qa.sqlite3 : AIA官網簡章html，加上47個QA集  
．chroma_t8_aia_with_qa.sqlite3 : 5個範例word簡章，加上47個QA集  

In [15]:
# 註解打開選一個 DB，需要與之後的 chromadb 的 collection相對應。 選好後直接shift+enter執行。

# choose_one = "chroma_t5.sqlite3"
# choose_one = "chroma_t6_with_qa.sqlite3"
# choose_one = "chroma_t8_aia_with_qa.sqlite3"
choose_one = "chroma_t13_aia_with_qa.sqlite3"


sqlite3_db_path = choose_one


class ChromaStore_sqlite3(BaseStore[str, bytes]):

    def __init__(self, path, table_name):
        self.connection = sqlite3.connect('{path}/{db_path}'.format(path=path, db_path=self.sqlite3_db_path))
        self.table = Table(table_name)
        self.id_column = Field('id')
        self.data_column = Field('data')
        self._create_table()

    def _create_table(self):
        id_column = Column('id', 'VARCHAR(50)', nullable=False)
        data_column = Column('data', 'VARCHAR(2500)', nullable=False)
        create_table_query = Query.create_table(self.table).columns(id_column, data_column).if_not_exists()
        cursor = self.connection.cursor()
        cursor.execute(create_table_query.get_sql())
        cursor.close()

    def mget(self, keys: Sequence[str]) -> List[Optional[bytes]]:
        cursor = self.connection.cursor()
        data_list = []

        for key in keys:
            
            key = [key]
            select_query = Query.from_(self.table).select(self.data_column).where(self.id_column.isin(key))
    
            cursor.execute(select_query.get_sql())
            results = cursor.fetchall()
    
            # Sorting to align with keys
            # sorted_results = sorted(results, key=lambda res: keys.index(json.loads((json.loads(json.loads(res[0]))["kwargs"]["page_content"]).replace("'", '"'))["id"]))
    
            for result in results:
                if result[0] is not None:
                    data_list.append(json.loads(result[0]).encode("utf-8"))
                else:
                    data_list.append(None)
                    
        cursor.close()

        return data_list
        
    def mset(self, key_value_pairs: Sequence[Tuple[int, bytes]]) -> None:
        insert_queries = []
        for key, value in key_value_pairs:
            insert_query = Query.into(self.table).columns(self.id_column, self.data_column).insert(key, json.dumps(value.decode('utf-8')))
            insert_queries.append(insert_query)

        cursor = self.connection.cursor()
        for query in insert_queries:
            cursor.execute(query.get_sql())

        self.connection.commit()

        cursor.close()

    def mdelete(self, keys: Sequence[int]) -> None:
        delete_query = Query.from_(self.table).delete().where(self.id_column.isin(keys))

        cursor = self.connection.cursor()
        cursor.execute(delete_query.get_sql())

        self.connection.commit()

        cursor.close()

    def yield_keys(self, prefix: Optional[str] = None) -> Iterator[str]:
        select_query = Query.from_(self.table).select(self.id_column)
        if prefix:
            select_query = select_query.where(self.id_column.like(f'{prefix}%'))

        cursor = self.connection.cursor()
        cursor.execute(select_query.get_sql())

        for row in cursor.fetchall():
            yield row[0]

        cursor.close()


ChromaStore_sqlite3.sqlite3_db_path = sqlite3_db_path

print('done')

done


In [16]:
# from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from typing import List

此處定義 embedding 模型，nomic-embed-text-v1 目前測試對於繁體中文支持度較差，須待測試更換其他模型。

In [17]:
# 8k token
class Nomic_Embed_V1:
        def __init__(self, model):
            self.model = SentenceTransformer(model, trust_remote_code=True)
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode([query]).tolist()[0]


# embeddings = Nomic_Embed_V1("nomic-ai/nomic-embed-text-v1")

print('done')

done


In [18]:
# 512 token
class Stella_Base_zh_v3_1792d:
        def __init__(self, model):
            self.model = SentenceTransformer(model, trust_remote_code=True)
    
        def embed_documents(self, texts: List[str]) -> List[List[float]]:
            return [self.model.encode(t).tolist() for t in texts]
        
        def embed_query(self, query: str) -> List[float]:
            return self.model.encode([query]).tolist()[0]


embeddings = Stella_Base_zh_v3_1792d("infgrad/stella-base-zh-v3-1792d")

print('done')

Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


done


In [9]:
# embeddings

定義 vectorestore

In [20]:
# 選定一個 ChromaDB 裡面的 collection，需要與先前的 sqlite3 db對應，例如 t5對應t5，t6對應t6，t8對應t8。

# collection_name = "simple_html_nomic_embed_text_v1_f16_t5"
# collection_name = "simple_html_nomic_embed_text_v1_f16_t6_with_qa"
# collection_name = "simple_html_nomic_embed_text_v1_f16_t8_aia_with_qa"
collection_name = "simple_html_Stella_Base_zh_v3_1792d_t13_aia_with_qa"


remote_vectorestore = Chroma(
    collection_name = collection_name,
    embedding_function=embeddings,
    client=httpClient
)

print('done')

done


定義 MultiVectorRetriever

In [21]:
# doc_store_path 指定為存放下載的 sqlite3 檔案的目錄
# top k 值 可更換

doc_store_path = r"C:\doc_store"


cs = ChromaStore_sqlite3(doc_store_path, "docstore")
store = create_kv_docstore(cs)

id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=remote_vectorestore, #remote_vectorestore
    docstore=store,
    id_key=id_key,
    search_kwargs={"k": 5}, # top k，invoke時招回的前 k 個相似的向量
)

print('done')

done


In [22]:
all_documents = remote_vectorestore.get()['documents'] # 查看目前 collection 內的檔案數據

print(len(all_documents))

94


In [23]:
# all_documents = remote_vectorestore.get()['documents'] # 查看目前 collection 內的檔案數據

# for doc in all_documents:
#     print(doc)
#     print('------------')

Langchain 的 MultiVectorRetirever，會先從 vectorstore 招回跟query相似的向量，然後經由id_key，返回docstore裡面對應的原始資料塊。  

我目前的做法是: 將原始資料切割後，存放在docstore裡面，並利用LLM模型(llama3:8b)對切割後的原始資料做主標題跟次標題的語意分析。  

使用下方的查詢，可以看到經由 vectorstore 返回的前 k 相似的招回。 上方的方法可以總攬 collection 內的所有檔案數據。  

目前的招回結果差強人意，等待測試更換其他的embedding模型看看回不會好一些。

In [24]:
test_data_retrieve = retriever.vectorstore.similarity_search("第四期LLM初階班開課時間", k=10)

for x in test_data_retrieve:
    print(x)

page_content='主標題：LLM初階班│第四期\n\n目錄：\n\n1. 獲錄取者註冊繳費規定\n2. 課前準備\n3. 課程大綱\n4. 專題討論\n5. 注意事項\n6. 退費辦法' metadata={'doc_id': '5a736f64e688-1'}
page_content='主標題：LLM初階班│第四期\n\n目錄：\n\n1. 辦公室流程自動化（RPA）\n2. 大型語言模型（LLM）\n3. RPA 與 LLM 螺合的優勢\n4. 課程成果\n5. 課程資訊\n6. 上課方式\n7. 適合對象\n8. 課程目標\n9. 課程內容\n10. 課程效益' metadata={'doc_id': '5a736f64e688-0'}
page_content='主標題：LLM進階班│第四期-招生簡章退費辦法更新版0427\n\n目錄：\n\n1. 課程大綱\n2. 報名網址\n3. 課程簡介\n4. 課程成果\n5. 課程資訊\n6. 課程目標\n7. 課程內容\n8. 課程效益\n9. 學費標準\n10. 報名方式' metadata={'doc_id': 'cca76c53cdfd-0'}
page_content='主標題: LLM進階班│第四期-招生簡章退費辦法更新版0427\n\n目錄:\n\n1. 報名繳費流程\n2. 獲錄取者註冊繳費通知\n3. 註冊繳費方式選擇\n4. 課前準備\n5. 課程大綱\n6. 課程照片\n7. 課程大綠週次內容\n8. 資料處理及連接\n9. LLM 開源模型微調方式\n10. 生成式 AI 產業應用' metadata={'doc_id': 'cca76c53cdfd-1'}
page_content='主標題：2024-LLM課程設計(做AI_四天)  地點更新版\n\n目錄：\n\n1. 課程概述\n2. 報名方式\n3. 入學考試\n4. 獲錄取通知及註冊繳費\n5. 註冊繳費辦法\n6. 注意事項' metadata={'doc_id': '04251beb763a-1'}
page_content='主標題：2024-LLM課程設計(做AI_四天)\n\n目錄：\n\n1. 報名資格\n2. 學費標準\n3. 校友優惠\n4. 團報優惠\n5. 報名方式\n6. 入學考試\n7. 放榜日

下方的查詢是 MultiVectorRetriever 真正所返回的存放在docstore裡面的原始文件切塊。 可以看到兩邊的 id 是對應的。  
top k 值無法參數指定，定義retriever時已指定。若要更換需要重新定義一次。

In [25]:
test_data_invoke = retriever.invoke("第四期LLM初階班開課時間")

for x in test_data_invoke:
    print(x)

page_content="{'id': '5a736f64e688-1', 'title': 'LLM初階班│第四期', 'text': '2. 獲錄取者需於收到錄取通知後 3 天內 完成註冊繳費。請於規定時間內辦理註冊及繳費，繳費方式可選擇線上金流 (刷卡) 或非線上金流 (轉帳)，若選擇非線上金流，系統會產生一組虛擬帳號，請務必在繳費期限內完成匯款繳費。繳費後才算完成報名程序。 3. 未依規定辦理或逾期未註冊者，取消入學資格，事後不得以任何理由要求補註冊。  ▍課前準備     1. 建議課前先使用gmail信箱註冊以下平台 ● OpenAI ● Make     2. 建議課前先安裝軟體 ● LM studio ( Mac 需 M1晶片以上) ● AnythingLLM     3. 請學員自備筆電，以便課程學習與小組專題實作  ▍前幾期小組成果發表主題 年節賀卡圖片生成推播 專案績效 AI 評分管理系統 旅遊行程規劃推播 法人金融報告流程自動化 課程滿意度問卷流程自動化 房屋建案資訊推播與問答 語音檔轉會議紀錄自動化 早安詩小話家 郵件內容條列整理推播 長輩圖新聞 Provider 可愛喵喵天氣推播 LINE 工作群請假/外出系統 快速依據圖片建立文案 AI 命理大師 在職進修課程推薦系統   ▍課程照片  \\n ▍課程大綱 週次 課程大綱 教學內容 第一週  ChatGPT 高效問答術 (Prompt engineering) ● 了解 LLM 的基本概念 ● 學習 LLM 的使用時機和方法 ● 讓 ChatGPT 角色扮演，得到更高品質的回應 ● 用模板和例子指導 ChatGPT，讓它執行你的任務 ○  Custom Instructions ● 用多個提示讓 ChatGPT 做更深的推理 ○  Few shot prompting ○  Chain of Thought prompt (思維鏈推理) ○  Chaining Prompts (多個提示鏈) ○  Tree-of Thought 辦公室流程自動化 ●  ChatGPT 輔助辦公室工作流程 ● 辦公室流程自動化實作  雲端、本機端LLM模型應用 ● 開源 LLM 模型介紹 ● 雲端聊天機器人體驗 ● 地端 LLM 模型應用 \\n專題討論 ● 專題討論 第二週 RPA應用程式介面 ●  